# *Pré-processamento*

Esta é uma etapa importante do processamento de dados, que envolve vários conceitos. No pré-processamento os dados podem ser "limpos", padronizados e transformados. Ainda no pré-processamento ocorre também a extração e seleção de características.

*Limpar* os dados se refere a remoção, ou correção, de dados que estejam imcompletos, corrompidos ou imprecisos.
*Padronizar* os dados se refere a remover dados *outliers*, deixar todas as possíveis medidas na mesma escala e unidade e normalizar os dados se necerrário.
*Transformar* os dados se refere a colocar os dados em um formato que favoreça futuras extrações de características ou análises.

## Carregamento dos dados

In [ ]:
# Carregando dados
import numpy as np

data = {}
# Carregando apenas os dados de s1
data['s1'] = np.load('./lib/data/converted/s1.npy')
print(f'{data["s1"].shape} - (classes, ensaios, canais, linhas)')

**Desafio**: Adapte o código acima para que ele carregue os dados de todos os voluntários do dataset. *Dica*: utilize f strings para conseguir iterar sobre as chaves dos dicionarios e sobre os nomes dos arquivos.

## Limpeza dos dados

Ao coletar dados EMG, estes tendem a vir com uma interferência da rede elétrica. A interferência é na frequência da rede elétrica. No Brasil a rede elétrica possui uma frequência de 60 Hz, gerando uma interferência de mesma frequência. Em outros países a frequênciada rede elétrica é de 50 Hz, gerando uma interferência nesta faixa. Essa interferência precisa ser removida dos dados, para realizar tal remoção utiliza-se um filtro *notch* da frequência da rede elétrica. A base da dados utilizada neste tutorial já passou por este processo antes de ser disponibilizada pelo autor, portanto não é necessário aplicar este filtro novamente sobre os dados aqui utilizados.

## Segmentação de dados

As características normalmente são extraidas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de se separar o dado em janelas recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações não sejam perdidas. Na imagem a seguir podemos observar um sinal EMG, abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas há uma sobreposição de tamanho *T*. 

![WINDOWS](lib\img\segmentation-of-sEMG-data.png)

Na base de dados sendo utilizada neste tutorial, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência, quando tais características serão aplicadas é necessário tranformar o dado para o domínio da frequência  então o dado também é transformado para o domínio da frequência, utilizando STFT, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [14]:
from scipy.signal import stft

step = 470
segment = 1024
chunks_time = {}
chunks_fft = {}

n_win = int((data['s1'].shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time['s1'] = np.array([data['s1'][:,:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 3, 0, 4)

# Janelas do dado no domínio da frequência
_, _, chunks_fft['s1'] = stft(data['s1'], fs=4000, nperseg=1024, noverlap=512)
chunks_fft['s1'] = np.swapaxes(chunks_fft['s1'], 3, 4)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time["s1"].shape} - (classes, ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_fft["s1"].shape} - (classes, ensaios, canais, janelas, linhas)')

Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (10, 6, 2, 41, 1024) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (10, 6, 2, 41, 513) - (classes, ensaios, canais, janelas, linhas)


**Desafio**: Adapte o código acima para percorrer todos os voluntários do dataset. Ao seguir implemente uma função para criar os segmentos que receba como parâmetro os dados que definem o tamanho e o passo da janela. Diferentes tamanhos de janelas e de sobreposições influenciam na extração de características, esta função será útil para testar diversas variações. *Dica*: observe que os dados para tamanho e passo da janela são diferentes entre o domínio do tempo e da frequência. Pode ser necessário a implementação de funções auxiliares e/ou a inclusão de mais parâmetros.